## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State
from oracle import Oracle

kill_processes()

## Create Plans

In [ ]:
gazebo_word_path = "simulators/gazebo/worlds/medical_delivery.world"  ##
target_box_center = (-85.263400, 56.518300, 0.593695, 0)

origin = (0, 0, 0)
offsets = [  # east, north, up, heading
    target_box_center,
    (0, -10, 0, 0),
    (5, -15, 0, 0),
    (10, -10, 0, 0),
]

n_vehicles = len(offsets)

homes = np.array([offset[:3] for offset in offsets])

altitudes = [5, 5, 5, 7]
plans = [Plan.hover(takeoff_alt=5) for alt in altitudes]

## Visualization Parameters

In [3]:
homes = np.array([offset[:3] for offset in offsets])
waypoints = {
    f"city_homes": {"pos": homes[1:], "color": "blue"},
    f"rural_home": {"pos": homes[[0]], "color": "green"},
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 10], ground=-0.05
)

## Choose Simulator

In [4]:
simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path=gazebo_word_path,
    models=n_vehicles * ["iris"],
    colors=n_vehicles * ["blue"],
    markers=waypoints,
)

Launch Simulator

In [5]:
uavs = simulator.launch()
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀


## Execute Plan

In [7]:
uavs[0].plan

🚀 <Plan '📋 hover'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️ MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  🚀 <Action '🛫 TAKEOFF'>
    🚀 <Step '🔹 takeoff'>
  🕓 <Action '🛩️ FLY'>

In [6]:
while orac.vehs:
    orac.gather_broadcasts()
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 1: ▶️ Plan Started: 📋 hover
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 2: ▶️ Plan Started: 📋 hover
Vehicle 2: ▶️ Action Started: 🔧 PREARM
Vehicle 2: ▶️ Step Started: Check disarmed
Vehicle 3: ▶️ Plan Started: 📋 hover
Vehicle 3: ▶️ Action Started: 🔧 PREARM
Vehicle 3: ▶️ Step Started: Check disarmed
Vehicle 4: ▶️ Plan Started: 📋 hover
Vehicle 4: ▶️ Action Started: 🔧 PREARM
Vehicle 4: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 2: ✅ Step Done: Check disarmed
Vehicle 3: ✅ Step Done: Check disarmed
Vehicle 4: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF
Vehicle 2: ▶️ Step Started: Check EKF
Vehicle 3: ▶️ Step Started: Check EKF
Vehicle 4: ▶️ Step Started: Check EKF
Vehicle 3: ✅ Step Done: Check EKF
Vehicle 3: ▶️ Step Started: Check GPS
Vehicle 4: ✅ Step Done: Check EKF
Vehicle 4: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check EKF
Vehicle 1: ▶️ Step Started: Check GPS
Vehi

KeyboardInterrupt: 

In [ ]:
uavs[0].plan

🚀 <Plan 'hover'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  🚀 <Action 'takeoff'>
    🚀 <Step 'takeoff'>
  🕓 <Action 'fly'>